# Machine Learning Model with The Data of Clinics in Lower Saxony Germany



![Machine Learning Workflow](https://miro.medium.com/max/2400/1*QV1rVgh3bfaMbtxueS-cgA.png)<br>
Foto:https://miro.medium.com/max/2400/1*QV1rVgh3bfaMbtxueS-cgA.png

#### Erforderliche Importe,die benötigt werden.

In [124]:
import pandas as pd
import numpy as np
import re
import pickle
#import csv

import textblob
from textblob import TextBlob
from textblob_de import TextBlobDE
from textblob_de import PatternParser
from textblob.sentiments import NaiveBayesAnalyzer

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from wordcloud import WordCloud, STOPWORDS

import matplotlib.pyplot as plt
import matplotlib as mpl
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib inline

from PIL import Image
import pickle

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import bar_chart_race as bcr
from pathlib import Path

In [125]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#from bokeh.io import output_notebook
#output_notebook()

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)
#import holoviews as hv
#import hvplot
#from hvplot import pandas
#hv.extension("bokeh")

##### Konvertieren csv-Daten in Pandas DataFrame

In [126]:
data = pd.read_csv(r'C:\Users\name\Desktop\WB\Abschlussproject\klinikbewertung.de\alldata\alldata_klinikandgoogle.csv', encoding='utf-8') #https://github.com/ablmz/Final-Project/tree/adem/klinikbewertung.de

#### Wir erstellen Dataframe

In [127]:
df = pd.DataFrame(data, columns= ['Name der Klinik', 'Textuelle Bewertung','Sternebewertung','Rating'])

In [128]:
#for q in range(result.shape[0]):
# create a list of our conditions
conditions = [
    (df['Sternebewertung'] >= 3 ),
    (df['Sternebewertung'] < 3 )
    ]
# create a list of the values we want to assign for each condition
values = [1, 0]

# create a new column and use np.select to assign values to it using our lists as arguments
df['Rating'] = np.select(conditions, values)

# display updated DataFrame
#df.head()

In [129]:
#df.dtypes

In [130]:
#df.info

## Preproccessing and Data Preparation

Obwohl wir die Daten als UTF-8 kodiert haben, sollen wir einige Falsche Wörter ändern.

In [131]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\name\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\name\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [132]:
# stemmer = SnowballStemmer("german")
# stop_words = set(stopwords.words("german"))

def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
        - some encoding errors 
    """
#     RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
#     RE_TAGS = re.compile(r"<[^>]+>")
#     RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
#     RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
#     if for_embedding:
#         # Keep punctuation
#         RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
#         RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

#     text = re.sub(RE_TAGS, " ", text)
#     text = re.sub(RE_ASCII, " ", text)
#     text = re.sub(RE_SINGLECHAR, " ", text)
#     text = re.sub(RE_WSPACE, " ", text)
    text= text.str.replace('\n','')
    text= text.str.replace('Ã¶','ö')
    text= text.str.replace('Ã\x84','Ä')
    text= text.str.replace('Ã¤','ä')
    text= text.str.replace('Ã¼','ü')
    text= text.str.replace('Ã\x9f','ß')
    text= text.str.replace('Ã\x9c','Ü')

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

# Daten Modell

In [133]:
#df

In [134]:
"""Compute unique word vector with frequencies
exclude very uncommon (<10 obsv.) and common (>=30%) words
use pairs of two words (ngram)
"""
vectorizer = TfidfVectorizer(
    analyzer="word", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
)
vectorizer.fit(df["Textuelle Bewertung"])

TfidfVectorizer(max_df=0.3, min_df=10, ngram_range=(1, 2))

An important parameter that needs explanation is the ngram_range. An ngram of one means that you look at each word separately. An ngram of two (or bigram) means that you take the preceding and following word into account as well. Thus, some context is added. This is helpful because then a model can learn that "good" and "not good" are different. In our case, in addition to using each word by itself we also add bigrams to make use of context. Let's see some of the created ngrams and their indices:

In [135]:
# Vector representation of vocabulary
word_vector = pd.Series(vectorizer.vocabulary_).sample(3, random_state=1)
print(f"Unique word (ngram) vector extract:\n\n {word_vector}")

Unique word (ngram) vector extract:

 müssen und        5606
stunden später    7258
über die          9091
dtype: int64


This creates a numeric representation of the ngrams in our corpus. We see that the vectorizer also uses bigrams in addition to single words. The word "müssen und" is represented by the number 5606, while the number 7258 stands for the bigram "stunden später" and so on.
This is only the first part of our text to numeric process. Before we can move on to transform each sentence to a vector of TF-IDF values, we need to prepare the data for the modeling part first.

### Modeling

To test the classification performance of our model, we will perform a cross validation. For that, we split our data into a training and a testing set. The former is used to train the model and the latter to evaluate its predictions:

In [136]:
# Sample data - 25% of data to test set
train, test = train_test_split(df, random_state=1, test_size=0.25, shuffle=True)

X_train = train["Textuelle Bewertung"]
Y_train = train["Rating"]
X_test = test["Textuelle Bewertung"]
Y_test = test["Rating"]
print(X_train.shape)
print(X_test.shape)

(3927,)
(1309,)


The training set consists of more than 3927 rows and the testing will be performed on more than 1309 observations. Now, that we have split our data we can transform the text data into its `TF-IDF` representation:

In [137]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)
X_train_vec.get_shape()

(3927, 9132)

We can see that each of our sentences is now represented by a vector of length 9132. It might be interesting to compare text and numeric representation:

In [138]:
# Compare original comment text with its numeric vector representation
print(f"Original sentence:\n{X_train[3:4].values}\n")
# Feature Matrix
features = pd.DataFrame(
    X_train_vec[3:4].toarray(), columns=vectorizer.get_feature_names()
)
nonempty_feat = features.loc[:, (features != 0).any(axis=0)]
print(f"Vector representation of sentence:\n {nonempty_feat}")

Original sentence:
['Absolut zufrieden mit dem Team , von dem ich am 30.09.2011 um 13.00 ambulant operiert wurde. Die Krankenschwester waren auch sehr gut. Danke!']

Vector representation of sentence:
          00        09        13      2011        30  absolut        am  \
0  0.185898  0.214235  0.196529  0.201964  0.161042  0.15359  0.092103   

   ambulant  auch sehr    danke       dem   dem ich  dem team  \
0  0.198253   0.171252  0.13431  0.173229  0.204667  0.237733   

   die krankenschwester    ich am  krankenschwester   mit dem  operiert  \
0              0.227242  0.197669          0.185069  0.131094  0.129453   

   operiert wurde  sehr gut      team  team von        um     um 13   von dem  \
0        0.245439  0.109134  0.127542  0.227242  0.098073  0.258141  0.194352   

      waren  waren auch  wurde die  zufrieden  zufrieden mit  
0  0.092489     0.22125   0.192297   0.130723       0.206097  


For this five word sentence, the vector of length  9132 contains mostly zeros. However, the indices representing the used words / ngrams are non empty. They include the value that TF-IDF assigned to them. In this particular case, "um 13" (at 13 o'Clock) has the largest weight meaning that it is relatively frequent in our sentence while not being very common in other sentences of our dataset.

Now, that we have prepared our features, we can start to train and evaluate models. For a binary classification task there are many options to chose from in scikit-learn. We will focus on the ones that are most promising. In my experience they are: Logistic Regression, Support Vector Classification (SVC), Ensemble Methods (Boosting, Random Forest) and Neural Networks (i.e. Multi Layer Perceptron or MLP in sklearn). We will compare these models and chose the most promising one:

In [139]:
# models to test
classifiers = [
    LogisticRegression(solver="sag", random_state=1),
    LinearSVC(random_state=1),
    RandomForestClassifier(random_state=1),
    XGBClassifier(random_state=1, use_label_encoder =False),
    MLPClassifier(
        random_state=1,
        solver="adam",
        hidden_layer_sizes=(12, 12, 12),
        activation="relu",
        early_stopping=True,
        n_iter_no_change=1,
    ),
]
# get names of the objects in list (too lazy for c&p...)
names = [re.match(r"[^\(]+", name.__str__())[0] for name in classifiers]
print(f"Classifiers to test: {names}")

Classifiers to test: ['LogisticRegression', 'LinearSVC', 'RandomForestClassifier', 'XGBClassifier', 'MLPClassifier']


In [140]:
%%time
# test all classifiers and save pred. results on test data
results = {}
for name, clf in zip(names, classifiers):
    print(f"Training classifier: {name}")
    clf.fit(X_train_vec, Y_train)
    prediction = clf.predict(X_test_vec)
    report = sklearn.metrics.classification_report(Y_test, prediction)
    results[name] = report

Training classifier: LogisticRegression
Training classifier: LinearSVC
Training classifier: RandomForestClassifier
Training classifier: XGBClassifier
[22:24:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training classifier: MLPClassifier
Wall time: 16.3 s


Training LogisticRegression and LinearSVC was very fast while the remaining classifiers were significantly slower. This has to do with their higher model complexity but can also greatly vary depending on the parameters used.
After having trained all our models on the train data and applying their prediction on the test data, we can judge their performance:

In [141]:
# Prediction results
for k, v in results.items():
    print(f"Results for {k}:")
    print(f"{v}\n")

Results for LogisticRegression:
              precision    recall  f1-score   support

           0       0.94      0.89      0.91       515
           1       0.93      0.96      0.95       794

    accuracy                           0.93      1309
   macro avg       0.93      0.93      0.93      1309
weighted avg       0.93      0.93      0.93      1309


Results for LinearSVC:
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       515
           1       0.95      0.94      0.94       794

    accuracy                           0.93      1309
   macro avg       0.93      0.93      0.93      1309
weighted avg       0.93      0.93      0.93      1309


Results for RandomForestClassifier:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       515
           1       0.92      0.93      0.93       794

    accuracy                           0.91      1309
   macro avg       0.91      0.91  

The output of sci-kit's classification_report provides us with several metrics. For unbalanced data sets accuracy is an inappropriate metric. Since it's value solely tells you how many cases have been properly classified, a high value can be achieved by always predicting the majority class. In contrast, the f1-score puts precision and recall of each class in relation to each other. As such, it is a more fine grained measure. We can use an aggregate version of it to have a single metric summarizing the performance of each model. For example, the weighted f1-score is an average of the f1-scores of both our classes taking the class distribution into account. On the other hand, the macro f1-score averages over class scores without weighing them. Consequently, we'll use that as we wish to give the same importance to both our classes. This is because even though bad grades are much more rare they also have a more severe impact.
All methods achieve impressive results predicting the good ratings class with f1-scores above 0.89. However, results for the bad ratings class are much lower and vary wildly.
Here, the ensemble methods, i.e. RandomForest and XGBoost, perform worst. However, with some more effort to tune their parameters they would probably fare significantly better.
As a general rule of thumb, logistic regressions deliver decent results in many different use cases. Moreover, they are simple to apply, robust and computationally efficient. macro f1-score we have three best score, MLP, LR and SVC over 0.92. In general, SVC is comparable in speed and simplicity to the logistic regression. In addition, it often performs very well in classification tasks as we can see here. In contrast, Neural Networks perform extremely well in all sorts of tasks but are also much more complex and slow all around. Because of this, we will stick to the LinearSVC classifier for now.

### Parameter tuning

We've learned that linear SVC is a solid model choice and delivers great results out of the box. Still, we might be able to improve upon those by taking a more guided approach to choosing parameters. To do so, we will compare different parameters for feature creation as well as modeling. We can achieve this by making use of the pipeline and grid search functionality in sci-kit learn.  
The `Pipeline` object encapsulates several processing steps into one. The last step in a pipeline must have a `fit()` functionality. The previous steps a `transform()` and `fit()` functionality. The `Pipeline` object itself has the same `fit()`, `transform()` and `predict()` methods as any other model in scikit-learn. Thus, we can streamline the whole process of feature creation, model fitting and prediction into one step.  
With `GridSearchCV` we can define parameter spaces for our functions. All parameter combinations will be evaluated against one another in a cross validation using a defined score metric. Next, we combine our pipeline with grid search. In result, we can jointly test combinations of parameters in feature creation and model training:

In [142]:
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8, 0.2),
    "tfidf__min_df": np.arange(5, 100, 45),
}
pipe_clf = GridSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro")
pipe_clf.fit(X_train, Y_train)
pickle.dump(pipe_clf, open("./clf_pipe.pck", "wb"))

Each value added to the `params` to be tested with `GridSearchCV` increases the run time of the training significantly. To limit the combinations, we first optimize the feature creation step. We test for different values for `ngram`. Moreover, `max_df` and `min_df` set an upper and lower limit for word frequencies. We want to exclude infrequent words because the model won't be able to learn (meaningful) associations with very few observations. The same is true for very frequent words which won't allow the model to differentiate between classes. For each parameter we check the values which returned the best model fit:

In [143]:
print(pipe_clf.best_params_)

{'tfidf__max_df': 0.3, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}


Now, using this best parameters for TF-IDF we can search for optimal parameters for the LinearSVC classifier:

In [144]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 3)],
    "tfidf__max_df": [0.5],
    "tfidf__min_df": [5],
    "svc__C": np.arange(0.2, 1, 0.15),
}
pipe_svc_clf = GridSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro")
pipe_svc_clf.fit(X_train, Y_train)


#pickle.dump(pipe_svc_clf, open("./pipe_svc_clf.pck", "wb")) first version

pickle.dump(pipe_svc_clf, open("./pipe_svc_clf.pkl", "wb"))

model = pickle.load(open("./pipe_svc_clf.pkl","rb"))

Wall time: 44 s


In [145]:
# best_params = pipe_svc_clf.best_params_
# print(best_params)

We focus on the C parameter which is basically a regularization parameter and essential in the performance of the SVC classifier. High values of C mean the margin of the hyperplane chosen by SVC to separate the data will be smaller. Thus, while classification on training data will be better this can also lead to overfitting. Consequently, C controls a trade off between a low training and testing error.

Finally, we combine these best parameters and test the prediction of our model using the pipe:

In [146]:
# run pipe with optimized parameters
pipe.set_params(**best_params).fit(X_train, Y_train)
pipe_pred = pipe.predict(X_test)
report = sklearn.metrics.classification_report(Y_test, pipe_pred)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.92      0.93       515
           1       0.95      0.95      0.95       794

    accuracy                           0.94      1309
   macro avg       0.94      0.94      0.94      1309
weighted avg       0.94      0.94      0.94      1309



Using parameter tuning we have slightly improved on our already decent model by increasing the precision for class 1.0. However, we can see that the margin for improvement is little. On one hand, this is because our initial parameters were already very close to the optimum. On the other hand, it might be that given our data and the approach used we might already be close to a barrier.

In [147]:
pickle.dump(pipe, open("./pipe.pkl", "wb"))
model = pickle.load(open("./pipe.pkl","rb"))

### Prediction

Now, that we have our best performing model (so far) we can use it to make predictions. One possible application is to find contradictory reviews, i.e. reviews where the sentiment of the comment doesn't match the rating. For that, we look at cases where our model makes a prediction with high confidence which doesn't match the original rating:

In [148]:
# # Get confidence score for prediction
# conf_score = pipe.decision_function(X_test)
# # Get the Nth highest / lowest score
# # high score indicates class 1 (bad), low score 0 (good)
# score_neg = np.sort(conf_score)[-1300]
# score_pos = np.sort(conf_score)[1300]
# # print(
# #     f"Threshold for negative rating: {score_neg}\nThreshold for positive rating: {score_pos}"
# # )

The `decision_function` returns values ranging from negative to positive (without a general boundary). They depict the distance of the input to the hyperplane which the SVM algorithm uses to separate the two classes. In our case, large values indicate a higher confidence in class 1 (i.e. a bad rating) and low values in class 0. Consequently, we take the n highest (lowest) values as a threshold for a high confidence in class 1 (0). 

In [149]:
#pd.options.display.max_colwidth = 800
# Predicted good but rated bad
#test[["Rating", "Textuelle Bewertung"]][(Y_test != pipe_pred) & (conf_score <= score_pos)]

In [150]:
# Predicted bad but rated good
#test[["Rating", "Textuelle Bewertung"]][(Y_test != pipe_pred) & ((conf_score >= score_neg))]

In [161]:
# Get new comments from website that were not included in original data
INPUT = [ "Nach meiner Beobachtung hat diese Praxis eine schlechte Hygiene."]
#          "Sprechstundenhilfe war super nett man fühlt sich wohl."
#          "Super sympathische Ärztin, fühle mich bei ihr bestens aufgehoben."
#         "nicht zuverlässig und nicht vertrauenswürdig."]
#          "Frau Doktor Merz nimmt sich richtig Zeit für mich. Hilft wo sie kann."
#          "Hört wirklich einen zu. Sehr nett und freundlich. Sie ist sehr kompetent,"
#          "zuverlässig und vertrauenswürdig.",
#          "Nach meiner Beobachtung hat diese Praxis eine schlechte Hygiene. ",
#          "Mangels akriebischer Behandlung musste mehrmals nachgebessert werden."
#          ]

In [162]:
# Pre-Process comments as we did with train data
#text = [clean_text(comment) for comment in INPUT]
#text = [comment for comment in INPUT]
text=INPUT
#text_out = " \n".join(text)
#print(f"Input after pre-processing / cleaning:\n\n{text_out}")
# run comments through pipe: predict using our best model from above
predictions = pipe.predict(text)

In [166]:
# Show comment and predicted Labels
predictions = pd.Series(predictions)
predictions = predictions.replace(0, "Schlechte Erfahrung").replace(1, "Gute Erfahrung")

# pd.concat(
#     [pd.Series(INPUT), predictions], axis="columns", keys=["Kommentar", "Hervorsage"]
# )

predictions[0]

'Schlechte Erfahrung'